In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
from keras import Model
from keras.layers import Dense , Flatten,Concatenate, Embedding,Input,Dropout
from keras.models import Sequential
from keras.regularizers import l1
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow import keras

seed = 2891  
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
from matplotlib import pyplot as plt
import warnings
import seaborn as sns 
warnings.filterwarnings('ignore')

C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_new = pd.read_csv('Train.csv')
band_names = [l.strip() for l in open('bandnames.txt', 'r').readlines()]
ss = pd.read_csv('SampleSubmission.csv')

In [3]:
def process_im(fid, folder='image_arrays_train'):
  fn = f'{folder}/{fid}.npy'
  arr = np.load(fn)
  bands_of_interest = ['S2_B1','S2_B2','S2_B3','S2_B4','S2_B5','S2_B6','S2_B7','S2_B8','S2_B8A','S2_B9','S2_B10',
                       'S2_B11','S2_B12','S2_QA10','S2_QA20' ,'S2_QA60','CLIM_aet','CLIM_def','CLIM_pdsi','CLIM_pet',
                       'CLIM_pr','CLIM_ro','CLIM_soil' ,'CLIM_srad' ,'CLIM_swe','CLIM_tmmn','CLIM_tmmx',
                       'CLIM_vap','CLIM_vpd','CLIM_vs']
  
  values = {}
  for month in range(12):
    bns = [str(month) + '_' + b for b in bands_of_interest] # Bands of interest for this month 
    idxs = np.where(np.isin(band_names, bns)) # Index of these bands
    vs = arr[idxs, 20, 20] # Sample the im at the center point
    for bn, v in zip(bns, vs[0]):
      values[bn] = v
  return values
def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['Yield'])
    y_predict = y_predict.join(t)   #Field_ID
    y_predic = y_predict[['Field_ID', 'Yield']]
    y_predic.to_csv(name_out + '.csv',index=False)
def input_fn_new(data_set, training = True):
    continuous_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
    
    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(data_set[k].size)], values = data_set[k].values, dense_shape = [data_set[k].size, 1]) for k in FEATURES_CAT}

    # Merges the two dictionaries into one.
    feature_cols = dict(list(continuous_cols.items()) + list(categorical_cols.items()))
    
    if training == True:
        # Converts the label column into a constant Tensor.
        label = tf.constant(data_set[LABEL].values)

        # Returns the feature columns and the label.
        return feature_cols, label
    
    return feature_cols
train_sampled = pd.DataFrame([process_im(fid) for fid in train_new['Field_ID'].values])

# Add in the field ID and yield
train_sampled['Field_ID'] = train_new['Field_ID'].values
train_sampled['Yield'] = train_new['Yield'].values
# train_sampled.head()
test_sampled = pd.DataFrame([process_im(fid, folder='image_arrays_test') for fid in ss['Field_ID'].values])
# test_sampled['Field_ID'] = ss['Field_ID']
# test_sampled['Yield'] = 0
# Example
# process_im('35AFSDD')

In [128]:
train = train_sampled.copy()
test = test_sampled.copy()
add_info = pd.read_csv('fields_w_additional_info.csv')

In [129]:
def NDVI(df):
  bands_of_interest = ['S2_B4','S2_B8']
  
  values = {}
  for month in range(12):
    df[str(month) + '_'+'NDVI'] = (df[str(month) + '_'+bands_of_interest[1]] - df[str(month) + '_'+bands_of_interest[0]]) / (df[str(month) + '_'+bands_of_interest[1]] + df[str(month) + '_'+bands_of_interest[0]])  
#     [str(month) + '_' + b for b in bands_of_interest] # Bands of interest for this month 
#     idxs = np.where(np.isin(band_names, bns)) # Index of these bands
#     vs = arr[idxs, 20, 20] # Sample the im at the center point
#     for bn, v in zip(bns, vs[0]):
#       values[bn] = v
    
  return df


In [130]:
train = NDVI(train)
test = NDVI(test)

In [76]:
# train
train['Quality'] = train_new.Quality
train = train[train.Quality>1]
train.drop('Quality', axis = 1, inplace = True)

In [131]:
# add_info.isna().sum()
# A=add_info[add_info.columns[9]]
# Anan=A[~np.isnan(A)] # Remove the NaNs
# sns.distplot(Anan)
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer = imputer.fit(add_info)
# X= imputer.transform(add_info)
add_info[add_info.columns.tolist()[1:]] = add_info[add_info.columns.tolist()[1:]].apply(lambda x: x.fillna(x.mean()),axis=0)
prepo_add_info = MinMaxScaler()
add_info[add_info.columns.tolist()[1:]] = prepo_add_info.fit_transform(add_info[add_info.columns.tolist()[1:]])
# add_info.isna().sum()
add_info_train = add_info.loc[add_info['Field_ID'].isin(train.Field_ID.values)]
add_info_test =  add_info.loc[add_info['Field_ID'].isin(ss.Field_ID.values)]

In [132]:
# train.Field_ID
# add_info.where(add_info.Field_ID == train.Field_ID.values)
# add_info_test
add_info_cols = add_info.columns.tolist()[1:]

In [133]:

# # Import and split
# # train = pd.read_csv('train_df_cust.csv')
# train_numerical = train.select_dtypes(exclude=['object'])
# train_categoric = train.select_dtypes(include=['object'])
# train = train_numerical.merge(train_categoric, left_index = True, right_index = True) 

# # test = pd.read_csv('test_df_cust.csv')
# Yield_ID = ss.Field_ID
# test_numerical = test.select_dtypes(exclude=['object'])
# test_categoric = test.select_dtypes(include=['object'])
# test = test_numerical.merge(test_categoric, left_index = True, right_index = True) 
# # Removie the outliers

# clf = IsolationForest(max_samples = 100, random_state = 42)
# clf.fit(train_numerical)
# y_noano = clf.predict(train_numerical)
# y_noano = pd.DataFrame(y_noano, columns = ['Top'])
# y_noano[y_noano['Top'] == 1].index.values

# train_numerical = train_numerical.iloc[y_noano[y_noano['Top'] == 1].index.values]
# train_numerical.reset_index(drop = True, inplace = True)

# train_categoric = train_categoric.iloc[y_noano[y_noano['Top'] == 1].index.values]
# train_categoric.reset_index(drop = True, inplace = True)

# train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)

In [134]:
test_numerical = test.select_dtypes(exclude=['object'])
test_categoric = test.select_dtypes(include=['object'])
train_numerical = train.select_dtypes(exclude=['object'])
train_categoric = train.select_dtypes(include=['object'])
col_train = list(train.columns)
col_train_bis = list(train.columns)
col_train_bis.remove('Yield')

col_train_num = list(train_numerical.columns)
col_train_num_bis = list(train_numerical.columns)

col_train_cat = list(train_categoric.columns)

col_train_num_bis.remove('Yield')

mat_train = np.matrix(train_numerical)
mat_test  = np.matrix(test_numerical)
mat_new = np.matrix(train_numerical.drop('Yield',axis = 1))
mat_y = np.array(train.Yield)

In [135]:
# mat_test[2].shape
# 
# (mat_new ==  ) .sum()
# mat_train.shape

In [137]:
prepro_y = MinMaxScaler()
prepro_y.fit(mat_y.reshape(2977,1))

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_test)

train_num_scale = pd.DataFrame(prepro.transform(mat_train),columns = col_train_num)
test_num_scale  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_num_bis)

train[col_train_num] = pd.DataFrame(prepro.transform(mat_train),columns = col_train_num)
test[col_train_num_bis]  = test_num_scale

In [138]:
# train['Yield']

In [139]:
# # List of features
# COLUMNS = col_train_num
# FEATURES = col_train_num_bis
# LABEL = "Yield"

# FEATURES_CAT = col_train_cat

# engineered_features = []

# for continuous_feature in FEATURES:
#     engineered_features.append(
#         tf.feature_column.numeric_column(continuous_feature))

# for categorical_feature in FEATURES_CAT:
#     sparse_column = tf.feature_column.categorical_column_with_hash_bucket(
#         categorical_feature, hash_bucket_size=1000)

#     engineered_features.append(tf.feature_column.embedding_column(categorical_column=sparse_column, dimension=16,combiner="sum"))
                                 
# # Training set and Prediction set with the features to predict
# training_set = train[FEATURES + FEATURES_CAT]
# prediction_set = train.Yield

# # Train and Test 
# x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES + FEATURES_CAT] ,
#                                                     prediction_set, test_size=0.33, random_state=42)
# y_train = pd.DataFrame(y_train, columns = [LABEL])
# training_set = pd.DataFrame(x_train, columns = FEATURES + FEATURES_CAT).merge(y_train, left_index = True, right_index = True)

# # Training for submission
# training_sub = training_set[FEATURES] # + FEATURES_CAT]
# testing_sub = test[FEATURES] # + FEATURES_CAT]

In [140]:
# # a[0]['Field_ID']
from keras.utils import to_categorical
# to_categorical(train['Field_ID'])

In [141]:
total = add_info['Field_ID'].values.copy()
par = train['Field_ID'].values.copy()
sar = ss.Field_ID.values


In [142]:
tot = par.tolist()+sar.tolist()

In [143]:
# par

In [144]:
# len(one_hot_encode[0])

In [145]:
mapping = {}
for x in range(len(tot)):
  mapping[tot[x]] = x

# integer representation
for x in range(len(tot)):
  tot[x] = mapping[tot[x]]


one_hot_encode = to_categorical(tot)
print(one_hot_encode)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [146]:
# mapping[ss.Field_ID[55]]
# one_hot_encode.shape
# len(par)
bad = train_new[train_new.Quality == 1]['Field_ID'].values.tolist()


In [147]:
# train[train.Field_ID in  bad]

In [148]:
no_of_unique_cat  = len(tot)
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
embedding_size = int(embedding_size)

In [149]:
num_data = train[col_train_num_bis]

In [161]:
keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)
inp_cat_data = Input(shape=(no_of_unique_cat,))
inp_num_data = Input(shape=(num_data.shape[1],))
input_add_data = Input(shape=(add_info_train[add_info_cols].shape[1],))
# Bind nulti_hot to embedding layer
emb = Embedding(input_dim=no_of_unique_cat, output_dim=embedding_size)(inp_cat_data)  
# Also you need flatten embedded output of shape (?,3,2) to (?, 6) -
# otherwise it's not possible to concatenate it with inp_num_data
flatten = Flatten()(emb)
# Concatenate two layers
conc = Concatenate()([flatten, inp_num_data])
conc2 = Concatenate()([conc, input_add_data])

# x = BatchNormalization()(conc)
dense1 = Dense(1148, activation='relu')(conc2)
x = Dropout(0.2)(dense1)
# x =  BatchNormalization()(dense1)
dense2 = Dense(512,activation = 'relu')(x)
x= Dropout(0.2)(dense2)
# dense21 = Dense(512,activation = 'relu')(x)
# x= Dropout(0.2)(dense21)
dense3 = Dense(256,activation = 'relu')(x)
x = Dropout(0.2)(dense3)
dense4 = Dense(128,activation = 'relu')(x)
# x = Dropout(0.2)(dense4)
# dense5 = Dense(64,activation = 'relu')(x)

# dense4 = Dense(128, activation='relu')(input_add_data)
# dense5 = Dense(512,activation = 'relu')(dense4)
# dense6 = Dense(128,activation = 'relu')(dense5)

# dense7 = Dense(32,activation = 'relu')(dense3)

out = Dense(1, activation='linear')(dense4)
model = Model(inputs=[inp_cat_data, inp_num_data, input_add_data], outputs=out)
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer='adam',loss='mse',metrics=['mean_squared_error'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min',restore_best_weights=True)
save_best = ModelCheckpoint('checkpoint', save_best_only=True, 
                               monitor='val_loss', mode='min')
model.compile(optimizer=opt,
              loss=keras.losses.Huber(delta=1),
              metrics=[keras.metrics.mean_squared_error])

In [162]:
# test
# try forest


In [163]:
# to_submit(pred_y,name_out)
# %load_ext tensorboard

import os
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)


In [164]:
hist = model.fit([one_hot_encode[:len(par)], train[col_train_num_bis], add_info_train[add_info_cols]] ,train['Yield'] , validation_split= 0.2,epochs=30,batch_size=100,verbose=1 , 
                 callbacks=[early_stopping,save_best, tensorboard_cb])
# %load_ext tensorboard
# %tensorboard --logdir=./my_logs --port=6006


Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
24/24 [==============================] - ETA: 0s - loss: 220.9701 - mean_squared_error: 1137919.6250WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001E2742E3288> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRA

In [154]:
# ss.Field_ID
# (train_sampled.Field_ID == 'E9UZCEA' ).any()
# train_sampled
# (train_new[train_new.Quality>1].Field_ID == 'E9UZCEA').any() 


# Model

In [155]:
# # training_set1[0]
# for x in range(len(sar)):
#    sar[x] = mapping[sar[x]]
# one_he = to_categorical(sar)

In [156]:
# add_info.Field_ID.shape

In [157]:
# train_sampled.shape

In [158]:
# ss.shape
y_predict = model.predict([one_hot_encode[len(par):], test[col_train_num_bis],add_info_test[add_info_cols]])


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [159]:
# to_categorical(sar)
# y_predict
ss = pd.read_csv('SampleSubmission.csv')
t = ss.Field_ID
to_submit(y_predict, "submission_continuous31")

In [160]:
chec = pd.read_csv('submission_continuous31.csv')
# chec['Yield'] = abs(chec['Yield'])
# (chec['Yield']<0).any()

In [127]:
# chec.to_csv('mod8_sub_abs.csv',index = False)
(chec['Yield']<0).any()

False

In [ ]:
prepro_y1 = MinMaxScaler()
prepro_y1.fit(mat_y.reshape(2642,1))
prepro_y1.inverse_transform(np.array(y_predict).reshape(len(y_predict),1))

In [ ]:
import seaborn as sns
(mat_y == 0).sum()

In [ ]:
# train.Yield
# chec = pd.read_csv('mod_sub_abs.csv')
# chec
chec

In [ ]:
A = train.merge(add_info_train, on = 'Field_ID')

In [ ]:
A.drop(['Field_ID', 'Yield'], axis = 1 , inplace = True)

In [ ]:
A.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 0.999)

In [ ]:
pca.fit(A)

In [ ]:
pca.components_.shape

In [ ]:
B = pca.transform(A)

In [ ]:
B.shape

In [ ]:
test['Field_ID'] = ss.Field_ID
C = test.merge(add_info_test, on = 'Field_ID')
C.drop(['Field_ID'], axis = 1 , inplace = True)
D = pca.transform(C)
D.shape


In [ ]:
# D[0].shape
# test.shape
229*6
# train
add_info_train.columns.tolist()